# **Task :** 
## **Number Recognition :**
### Handwritten  digit  recognition  system  not  only  detectsscanned  images  of  handwritten  digits.Handwritten  digitrecognition using MNIST dataset is a major project made with the help of Neural Network. It basically detectsthe scanned images of handwritten digits. 

### Import necessary libraries

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical


### Load the data

In [3]:
# Load the MNIST dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Preprocess the data
train_images = train_images.reshape((60000, 28, 28, 1)).astype("float32") / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype("float32") / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)


### Build the CNN model

In [4]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(10, activation="softmax"))


### Compile the model

In [5]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


### Train the model

In [6]:
model.fit(train_images, train_labels, epochs=5, batch_size=64, validation_split=0.2)


Epoch 1/5
750/750 [==============================] - 11s 6ms/step - loss: 0.2154 - accuracy: 0.9336 - val_loss: 0.0696 - val_accuracy: 0.9806
Epoch 2/5
750/750 [==============================] - 6s 8ms/step - loss: 0.0577 - accuracy: 0.9818 - val_loss: 0.0483 - val_accuracy: 0.9855
Epoch 3/5
750/750 [==============================] - 6s 8ms/step - loss: 0.0400 - accuracy: 0.9871 - val_loss: 0.0516 - val_accuracy: 0.9846
Epoch 4/5
750/750 [==============================] - 6s 8ms/step - loss: 0.0306 - accuracy: 0.9905 - val_loss: 0.0448 - val_accuracy: 0.9873
Epoch 5/5
750/750 [==============================] - 6s 8ms/step - loss: 0.0253 - accuracy: 0.9918 - val_loss: 0.0475 - val_accuracy: 0.9865


### Evaluate the Model

In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc}")


313/313 [==============================] - 1s 5ms/step - loss: 0.0386 - accuracy: 0.9870
Test accuracy: 0.9869999885559082


### Save the Model

In [8]:
model.save("digit_recognition_model.h5")


## Creating a canvas for drawing & recognize

In [9]:
import tkinter as tk
from tkinter import Canvas, Button, Label
from PIL import Image, ImageDraw
import numpy as np
from tensorflow.keras.models import load_model


In [10]:
class DigitRecognizerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Digit Recognizer")

        self.canvas = Canvas(root, width=280, height=280, bg="white", cursor="cross")
        self.canvas.pack()

        self.label_result = Label(root, text="Prediction: ", font=("Helvetica", 16))
        self.label_result.pack()

        self.clear_button = Button(root, text="Clear", command=self.clear_canvas)
        self.clear_button.pack()

        self.predict_button = Button(root, text="Predict", command=self.predict_digit)
        self.predict_button.pack()

        # Load the trained model
        self.model = load_model("digit_recognition_model.h5")

        # Initialize drawing
        self.image = Image.new("L", (280, 280), color=255)
        self.draw = ImageDraw.Draw(self.image)
        self.old_x = None
        self.old_y = None

        # Set up canvas events
        self.canvas.bind("<B1-Motion>", self.paint)
        self.canvas.bind("<ButtonRelease-1>", self.reset)

    def paint(self, event):
        x, y = event.x, event.y
        if self.old_x and self.old_y:
            pen_width = 20
            self.canvas.create_line(self.old_x, self.old_y, x, y, width=pen_width, fill="black", capstyle=tk.ROUND, smooth=tk.TRUE)
            self.draw.line([self.old_x, self.old_y, x, y], fill=0, width=10)
        self.old_x = x
        self.old_y = y

    def reset(self, event):
        self.old_x = None
        self.old_y = None

    def clear_canvas(self):
        self.canvas.delete("all")
        self.image = Image.new("L", (280, 280), color=255)
        self.draw = ImageDraw.Draw(self.image)
        self.label_result.config(text="Prediction: ")

    def predict_digit(self):
        img = self.image.resize((28, 28), Image.BICUBIC)
        img = np.array(img) / 255.0
        img = img.reshape(1, 28, 28, 1)

        prediction = self.model.predict(img)
        predicted_digit = np.argmax(prediction)

        self.label_result.config(text=f"Prediction: {predicted_digit}")

In [11]:
root = tk.Tk()
app = DigitRecognizerApp(root)
root.mainloop()

## Creating GUI for Scanned Images & Recognize

In [12]:
import tkinter as tk
from tkinter import Button, Label, filedialog
from PIL import Image, ImageTk
import numpy as np
from tensorflow.keras.models import load_model

class ScannedImagePredictorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Digit Recognizer - Scanned Image Predictor")

        self.label_result = Label(root, text="Prediction: ", font=("Helvetica", 16))
        self.label_result.pack()

        self.load_button = Button(root, text="Load Image", command=self.load_image)
        self.load_button.pack()

        self.predict_button = Button(root, text="Predict", command=self.predict_digit)
        self.predict_button.pack()

        self.image_label = Label(root)
        self.image_label.pack()

        # Load the trained model
        self.model = load_model("digit_recognition_model.h5")

        # Initialize variables
        self.image_path = None

    def load_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.gif;*.bmp")])
        if file_path:
            self.image_path = file_path
            image = Image.open(self.image_path)
            image = image.resize((280, 280), Image.BICUBIC)
            self.photo = ImageTk.PhotoImage(image)
            self.image_label.config(image=self.photo)

    def predict_digit(self):
        if self.image_path:
            img = Image.open(self.image_path).convert("L")
            img = img.resize((28, 28), Image.BICUBIC)
            img_array = np.array(img) / 255.0
            img_array = img_array.reshape(1, 28, 28, 1)

            prediction = self.model.predict(img_array)
            predicted_digit = np.argmax(prediction)

            self.label_result.config(text=f"Prediction: {predicted_digit}")
        else:
            self.label_result.config(text="Please load an image first.")

if __name__ == "__main__":
    root = tk.Tk()
    app = ScannedImagePredictorApp(root)
    root.mainloop()
